# Loading the data From the Local system

In [ ]:
import pandas as pd
df=pd.read_csv('Concrete_Data.csv')
df.head(10)

In [ ]:
df.columns

# EDA

Scatterplot:

In [ ]:
import seaborn as sns
sns.scatterplot(x="Age",y="CMS",data=df)

It shows that in the starting the strength is weak but it solidifies with the time and again it loses its strength

In [ ]:
sns.scatterplot(x="FA",y="CMS",data=df)

The outcome shows with the density of 750 to 900 fine aggregate,maximum compressive strength lies

In [ ]:
sns.scatterplot(x="CA",y="CMS",data=df)

The outcome shows with the density of 950 to 1100 fine aggregate,maximum compressive strength lies

In [ ]:
sns.scatterplot(x="Superplasticizer",y="CMS",data=df)

The outcome shows with the density of 5 to 12 fine Superplasticizer,maximum compressive strength lies

In [ ]:
sns.scatterplot(x="Water",y="CMS",data=df)

The outcome shows with the density of 160 to 200 fine Superplasticizer,maximum compressive strength lies

In [ ]:
sns.scatterplot(x="Fly Ash",y="CMS",data=df)

The outcome shows with the density of 100 to 175 of Fly Ash,maximum compressive strength lies

In [ ]:
sns.scatterplot(x="Blast",y="CMS",data=df)

The outcome shows with the density of 0 to 200 of Blast,maximum compressive strength lies

In [ ]:
sns.scatterplot(x="Cement",y="CMS",data=df)

The outcome shows with the density of 100 to 400 of Blast,maximum compressive strength lies

In [ ]:
df.corr()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(15,9))
sns.heatmap(df.corr(),annot=True,linewidths=0.5,linecolor="black",fmt='.2f')  #fmt .2f for two decimal points

The positively correlated columns has a great impact on the target column while the negative correlated has less or zero impact on the target column

In [ ]:
plt.figure(figsize=(15,12))
sns.heatmap(round(df.describe()[1:].transpose(),2),linewidth=2,annot=True,fmt="f")
plt.xticks(fontsize=18)
plt.yticks(fontsize=12)
plt.title("Variables summary")
plt.show()

In [ ]:
df.describe()

# Now use subplot and displot to check data are normalized or not

In [ ]:
import itertools
cols=['Cement', 'Blast', 'Fly Ash', 'Water', 'Superplasticizer', 'CA', 'FA',
       'Age', 'CMS']
length=len(cols)
cs=["b","r","g","c","m","k","lime"]
fig=plt.figure(figsize=(13,25))

for i,j,k in itertools.zip_longest(cols,range(length),cs):
    plt.subplot(4,2,j+1)
    ax=sns.distplot(df[i],color=k,rug=True)
    ax.set_facecolor("w")
    plt.axvline(df[i].mean(),linestyle="dashed",label="mean",color="k")
    plt.legend(loc="best")
    plt.title(i,color="navy")
    plt.xlabel("")

# Observation:

from the above observation it is clear that the data are not normalized and we should proceed with the data cleaning method

In [ ]:
df.hist(edgecolor="red",linewidth=1.5,figsize=(20,20))

In [ ]:
plt.figure(figsize=(10,7))
df.corr()['CMS'].sort_values(ascending=False).drop(['CMS'])
plt.xlabel('Feature',fontsize=14)
plt.ylabel("Correlation with target column",fontsize=14)
plt.title("Correlation of features with the target column", fontsize=18)
plt.show()

the bara goes upper of 0 is positively correlated. Goes down 0 is nagetavely correlated

In [ ]:
df.isnull().sum()

In [ ]:
df.skew()

From the above observation the columns falls under high skewness :
    
    Age,Superplasticizer,Blast
    
But the columns have good correlation with the target column. Dropping of such column should not be good approach    

# Data Transformation:

we need to transform those columns which has skewness.

# Outliers Check:

In [ ]:
df["Cement"].plot.box()

In [ ]:
df["Blast"].plot.box()

In [ ]:
df["Fly Ash"].plot.box()

In [ ]:
df["Water"].plot.box()

In [ ]:
df["Superplasticizer"].plot.box()

In [ ]:
df["CA"].plot.box()

In [ ]:
df["FA"].plot.box()

In [ ]:
df["Age"].plot.box()

In [ ]:
df["CMS"].plot.box()

# Check the percentage of data falls under outliers:
    

In [ ]:
from scipy.stats import zscore
import numpy as np
z=np.abs(zscore(df))
threshold=3
np.where(z>3)

In [ ]:
df_new=df[(z<3).all(axis=1)]

In [ ]:
df_new

In [ ]:
df.shape

In [ ]:
df_new.shape

# Percentage loss of data while removing outliers:

In [ ]:
percentage_loss=((49)/1030)*100   #1030-981=49
print(percentage_loss)

Since the percentage of outliers are less than 5 percent.We can proceed with the outliers removable

Sometimes z-score unable to remove outliers.So we prefer IQR here over z-score

In [ ]:
q1=df.quantile(0.25)
q3=df.quantile(0.75)
IQR=q3-q1
print(IQR)

In [ ]:
df_new1=df[~((df<(q1-1.5*IQR))|(df>(q3+1.5*IQR))).any(axis=1)]
print(df_new1.shape)

In [ ]:
percentage_loss=((1030-941)/1030)*100
print(percentage_loss)

The percentage loss in the z-score is less so we prefer z-score over IQR

In [ ]:
x=df_new.iloc[:,:-1]
y=df_new.iloc[:,-1]

# Removing skewnes by transforming data

In [ ]:
from sklearn.preprocessing import power_transform     #box-cox is only for positive but yeo-johnson is for both negative and positive
x=power_transform(x,method='yeo-johnson')

# Regression Model Building:

# Linear Regression:

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_t=sc.fit_transform(x)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
from sklearn.metrics import r2_score

In [ ]:
max_scr=0
for i in range(0,1000):
    x_train,x_test,y_train,y_test=train_test_split(x_t,y,test_size=.20,random_state=i)
    lr.fit(x_train,y_train)
    pred_train=lr.predict(x_train)
    pred_test=lr.predict(x_test)
    if round(r2_score(y_train,pred_train)*100,2)==round(r2_score(y_test,pred_test)*100,2):
        print("At random state",i,"The model performs very well")
        print("At random state :-",i)
        print("Training r2_score is :-",r2_score(y_train,pred_train)*100)
        print("Testing r2_score is :-",r2_score(y_test,pred_test)*100)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x_t,y,test_size=.20,random_state=96)

We will use random state 96

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')

# Regularization:

# 1. Lasso:

In [ ]:
from sklearn.linear_model import Lasso

parameters={'alpha':[.0001,.001,.01,.1,1,10],'random_state':list(range(0,10))}
ls=Lasso()
clf=GridSearchCV(ls,parameters)
clf.fit(x_train,y_train)

print(clf.best_params_)

In [ ]:
ls=Lasso(alpha=.0001,random_state=0)
ls.fit(x_train,y_train)
ls.score(x_train,y_train)
pred_ls=ls.predict(x_test)

lss=r2_score(y_test,pred_ls)
for j in range (2,10):
    lsscore=cross_val_score(ls,x_t,y,cv=j)
    lsc=lsscore.mean()
    print("At cv:-",j)
    print("Cross Validation Score is :- ", lsc*100)
    print("R2 score is :-", lss*100)
    print("\n")

In [ ]:
print("At cv:-",3)
print("Cross Validation Score is :- ", 71.650)
print("R2 score is :-", 81.210)

The value of cv should be 3

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import numpy as np

In [ ]:
print("Error")
print("Mean Absolute Eror : ",mean_absolute_error(y_test,pred_ls))
print("Mean Squared Eror : ",mean_squared_error(y_test,pred_ls))
print("Root Mean Squared Eror : ",np.sqrt(mean_squared_error(y_test,pred_ls)))

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(x=y_test,y=pred_ls,color='r')
plt.plot(y_test,y_test,color='b')
plt.xlabel("Actual compressive strength",fontsize=14)
plt.ylabel("Predicted compressive strength",fontsize=14)
plt.title('Lasso Regression',fontsize=18)
plt.show()


The above results shows that the datapoints near the best fit line. Trying model testing with different algorithm

# 2.Ridge Regression

In [ ]:
from sklearn.linear_model import Ridge

parameters={'alpha':[.0001,.001,.01,.1,1,10],'copy_X':[True,False],'random_state':list(range(0,10)),'fit_intercept':[True,False],'normalize':[True,False],'tol':[.0001,.001,.01,.1,1,10]}
rd=Ridge()
clf=GridSearchCV(rd,parameters)
clf.fit(x_train,y_train)

print(clf.best_params_)

In [ ]:
rd=Ridge(alpha=0.01, copy_X= True, fit_intercept= True, normalize= True, random_state= 0, tol= 0.0001)
rd.fit(x_train,y_train)
rd.score(x_train,y_train)
pred_rd=rd.predict(x_test)

rds=r2_score(y_test,pred_rd)
print('R2 Score:',rds*100)

rdscore=cross_val_score(rd,x_t,y,cv=3)
rdc=rdscore.mean()
print('Cross Val Score : ',rdc*100)

In [ ]:
print("Error")
print("Mean Absolute Eror : ",mean_absolute_error(y_test,pred_rd))
print("Mean Squared Eror : ",mean_squared_error(y_test,pred_rd))
print("Root Mean Squared Eror : ",np.sqrt(mean_squared_error(y_test,pred_rd)))

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(x=y_test,y=pred_rd,color='r')
plt.plot(y_test,y_test,color='b')
plt.xlabel("Actual compressive strength",fontsize=14)
plt.ylabel("Predicted compressive strength",fontsize=14)
plt.title('Ridge Regression',fontsize=18)
plt.show()

# Ensemble Techniques : 

Decision Tree Regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor

parameters={'criterion':['mse','friedman_mse','mae'],'splitter':['best','random']}
dt=DecisionTreeRegressor()
clf=GridSearchCV(dt,parameters)
clf.fit(x_train,y_train)

print(clf.best_params_)

In [ ]:
dt=DecisionTreeRegressor(criterion= 'mse', splitter= 'best')
dt.fit(x_train,y_train)
dt.score(x_train,y_train)
pred_dt=dt.predict(x_test)

dts=r2_score(y_test,pred_dt)
print('R2 Score:',dts*100)

dtscore=cross_val_score(dt,x_t,y,cv=3)
dtc=dtscore.mean()
print('Cross Val Score : ',dtc*100)

In [ ]:
print("Error")
print("Mean Absolute Eror : ",mean_absolute_error(y_test,pred_dt))
print("Mean Squared Eror : ",mean_squared_error(y_test,pred_dt))
print("Root Mean Squared Eror : ",np.sqrt(mean_squared_error(y_test,pred_dt)))

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(x=y_test,y=pred_dt,color='r')
plt.plot(y_test,y_test,color='b')
plt.xlabel("Actual compressive strength",fontsize=14)
plt.ylabel("Predicted compressive strength",fontsize=14)
plt.title('Decision Tree Regression',fontsize=18)
plt.show()

Random Forest Regressor : 

In [ ]:
from sklearn.ensemble import RandomForestRegressor

parameters={'criterion':['mse','friedman_mse','mae'],'n_estimators':[100,200,300]}  #Search in google for more parameter
rf=RandomForestRegressor()
clf=GridSearchCV(rf,parameters)
clf.fit(x_train,y_train)

print(clf.best_params_)

In [ ]:
rf=RandomForestRegressor(criterion= 'mae', n_estimators= 200)
rf.fit(x_train,y_train)
rf.score(x_train,y_train)
pred_rf=rf.predict(x_test)

rfs=r2_score(y_test,pred_rf)
print('R2 Score:',rfs*100)

rfscore=cross_val_score(rf,x_t,y,cv=3)
rfc=rfscore.mean()
print('Cross Val Score : ',rfc*100)

In [ ]:
print("Error")
print("Mean Absolute Eror : ",mean_absolute_error(y_test,pred_rf))
print("Mean Squared Eror : ",mean_squared_error(y_test,pred_rf))
print("Root Mean Squared Eror : ",np.sqrt(mean_squared_error(y_test,pred_rf)))

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(x=y_test,y=pred_rf,color='r')
plt.plot(y_test,y_test,color='b')
plt.xlabel("Actual compressive strength",fontsize=14)
plt.ylabel("Predicted compressive strength",fontsize=14)
plt.title('Random Forest Regression',fontsize=18)
plt.show()

Gradient Boosting Regressor

In [ ]:
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingRegressor

parameters={'loss':['ls', 'lad', 'huber', 'quantile'],'n_estimators':[50,100,200],'criterion':['friedman_mse', 'mse', 'mae']}  #Search in google for more parameter
gbr=GradientBoostingRegressor()
clf=GridSearchCV(gbr,parameters)
clf.fit(x_train,y_train)

print(clf.best_params_)

In [ ]:
gbr=GradientBoostingRegressor()
gbr.fit(x_train,y_train)
gbr.score(x_train,y_train)
pred_gbr=gbr.predict(x_test)

gbrs=r2_score(y_test,pred_gbr)
print('R2 Score:',gbrs*100)

gbrscore=cross_val_score(gbr,x_t,y,cv=3)
gbrc=gbrscore.mean()
print('Cross Val Score : ',gbrc*100)

In [ ]:
print("Error")
print("Mean Absolute Eror : ",mean_absolute_error(y_test,pred_gbr))
print("Mean Squared Eror : ",mean_squared_error(y_test,pred_gbr))
print("Root Mean Squared Eror : ",np.sqrt(mean_squared_error(y_test,pred_gbr)))

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(x=y_test,y=pred_gbr,color='r')
plt.plot(y_test,y_test,color='b')
plt.xlabel("Actual compressive strength",fontsize=14)
plt.ylabel("Predicted compressive strength",fontsize=14)
plt.title('Gradient Boosting Regressor',fontsize=18)
plt.show()

SVR

In [ ]:
from sklearn.svm import SVR

parameters={'kernel':['linear', 'poly', 'rbf', 'sigmoid'],'gamma':['scale','auto']}  #'gamma':['scale','auto']
sv=SVR()
clf=GridSearchCV(sv,parameters)
clf.fit(x_train,y_train)

print(clf.best_params_)

In [ ]:
sv=SVR(kernel= 'linear',gamma='scale')
sv.fit(x_train,y_train)
sv.score(x_train,y_train)
pred_sv=sv.predict(x_test)

svs=r2_score(y_test,pred_sv)
print('R2 Score:',svs*100)

svscore=cross_val_score(sv,x_t,y,cv=3)
svc=svscore.mean()
print('Cross Val Score : ',svc*100)

In [ ]:
print("Error")
print("Mean Absolute Eror : ",mean_absolute_error(y_test,pred_sv))
print("Mean Squared Eror : ",mean_squared_error(y_test,pred_sv))
print("Root Mean Squared Eror : ",np.sqrt(mean_squared_error(y_test,pred_sv)))

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(x=y_test,y=pred_sv,color='r')
plt.plot(y_test,y_test,color='b')
plt.xlabel("Actual compressive strength",fontsize=14)
plt.ylabel("Predicted compressive strength",fontsize=14)
plt.title('SVR',fontsize=18)
plt.show()

# The best model is Lasso.Since the difference between the percentage score of cross validation and r2_score is optimum

# Model Saving

In [ ]:
import pickle
filename='inhouse_concrete.pkl'
pickle.dump(ls,open(filename,'wb'))

In [ ]:
import pickle

In [107]:
with open('inhouse_concrete.pkl','rb') as f:
    x=pickle.load(f)

# Conclusion:

In [ ]:
import numpy as np
a=np.array(y_test)
a

In [ ]:
predicted=np.array(ls.predict(x_test))
predicted

In [ ]:
df_com=pd.DataFrame({"original":a,"predicted":predicted},index=range(len(a)))
df_com

# From the above table the model is predicted the values with 70-80 percent accuracy

We can get all the datasets in UCI repository